
__BUILD A HMM FOR THE REFERENCE ALIGNMENT__

We will do this with the program `hmmbuild` from the [hmmer v3](http://hmmer.janelia.org/) program suite.

In [ ]:
!hmmbuild -h #for help

In [ ]:
!hmmbuild Notostraca_ref.hmm data/MT-CO1@mafft_reduced_Defaults.fasta

__ALIGN QUERIES TO REFERENCE ALIGNMENT__

We will do this with the program `hmmalign` from the hmmerv3 program suite.

In [ ]:
!hmmalign -h #for help

In [ ]:
!hmmalign -o Notostraca_ref_plus_random_query.sto --mapali data/MT-CO1@mafft_reduced_Defaults.fasta Notostraca_ref.hmm data/Notostraca_random_queries.fasta

__Create reference package required by pplacer__

First we will need to compile some information about the taxonomy of the reference sequences.

We start by producing a taxonomy table for the set of taxa that is used as reference. The file `taxids.txt` is a simple text file that contains the taxonomic ids [taxids](http://www.ncbi.nlm.nih.gov/taxonomy) for all taxa. 

In [ ]:
!cat data/taxids.txt

We will use a tool from the [taxtastic](http://fhcrc.github.io/taxtastic/) package to fetch the taxonomic information for these taxa from the global [NCBI taxonomy](http://www.ncbi.nlm.nih.gov/taxonomy), which is present as the so-called 'taxonomy dump' in our container (`/usr/bin/taxonomy.db`). Information on how to format the taxonomy dump for use with taxtastic can be found [here](http://fhcrc.github.io/taxtastic/commands.html#new-database).


[Taxtastic](http://fhcrc.github.io/taxtastic/) is suite of tools.

In [ ]:
!taxit -h #for help

To explore individual functions (such as `taxtable`) further do, e.g.:

In [ ]:
!taxit taxtable -h

Now, let's produce the relevant taxtable.

In [ ]:
!taxit taxtable -d /usr/bin/taxonomy.db -t data/taxids.txt -o taxa.csv

The resulting `taxa.csv` file contains just the taxonomic information relevant for the reference sequences to be used for the phylogenetic placement. 

Have a look into the file.



In [ ]:
!head taxa.csv

We will also need to provide information that links the taxonomic ids to the actual sequence ids. This file is called the 'seqinfo' file by taxtasic. We provide this as `seq_info.csv`. Take a second to think about how you would go about to extract this information from a genbank file that contains the complete information for the reference sequences (`Notostraca_COI_without_random.gb`).

Have a look:

In [ ]:
!head seq_info.csv

In [ ]:
!head Notostraca_COI_without_random.gb

The reference package also needs to contain a reference tree, the log from the tree inference, the underlying alignment in fasta format as well as the HMM profile that you have produced above to align the query sequences to. 

Compile the reference package as below:

In [ ]:
!taxit create -l COI -P Notostraca_COI.refpkg --aln-fasta MT-CO1@mafft_reduced_Defaults.fasta --tree-stats RAxML_info.MT-CO1@mafft_reduced_Defaults --tree-file RAxML_bestTree.MT-CO1@mafft_reduced_Defaults --profile Notostraca_ref.hmm --seq-info seq_info.csv --taxonomy taxa.csv

Some explanation for the above command:
    
```bash
!taxit create \ #call the relevant function
-l COI \ #id of the marker used (arbitrary)
-P Notostraca_COI.refpkg \ #name to be given to the reference package (arbitrary)
--aln-fasta MT-CO1@mafft_reduced_Defaults.fasta \ #the alignment underlying the tree
--tree-stats RAxML_info.MT-CO1@mafft_reduced_Defaults \ #log file from tree inference
--tree-file RAxML_bestTree.MT-CO1@mafft_reduced_Defaults \ #the tree in newick format
--profile Notostraca_ref.hmm \ #HMM profile build for your reference alignment
--seq-info seq_info.csv \ #seqinfo mapping taxonomy to sequence ids
--taxonomy taxa.csv \ #the relevant subset of the NCBI taxonomy
```

__PHYLOGENETIC PLACEMENT USING PPLACER__

Some more info on pplacer is [here](http://matsen.fhcrc.org/pplacer/). The approach provides a lot of options, some of which are a bit cryptic to be honest. 

See for yourself:



In [ ]:
!pplacer -h 

For the time being we will limit ourselfs to a basic analysis.

We'll provide the referenc package that we have produced and the query alignment. Further details:
```bash
!pplacer \ #call pplacer
-c Notostraca_COI.refpkg/ \ #point to refpkg
Notostraca_ref_plus_random_query.sto \ # profile alignment
-p \ #report posterior propabilities
--keep-at-most 3 \ #keep at most 3 assignments per query
-o queries.jplace \ #write output to this file
```

In [ ]:
!pplacer -c Notostraca_COI.refpkg/ Notostraca_ref_plus_random_query.sto -p --keep-at-most 3 -o queries.jplace

`pplacer` has its own extensive format to describe placements ('the placement file'). Have a look inside and if you want. It also provides a tool for the (comparative) analyis of placement files called `guppy` ('Grand Unified Phylogenetic Placement Yanalyzer'). Again, this is a tool with extensive functionality (see [here](http://matsen.github.io/pplacer/generated_rst/guppy.html) that is beyond the scope of this introductionary course.

However, because pplacer has its own format that can only be analysed by `guppy` and other tools from the `pplacer` suite, we provide a script that converts place files to the standardized [biom format](http://biom-format.org/), which can be interpreted by a number of tools, including [qiime](http://qiime.org/).



In [ ]:
!jplace_to_biom.py -h

Convert the placefile to [biom](http://biom-format.org/) format like so:


In [ ]:
!jplace_to_biom.py -p pplace -j queries.jplace 

This produces 2 files `pplace.biom` and `pplace.tsv`. The latter is a a simple tab-delimited file that you can open with any txt editor or e.g. excel.

`pplace.biom` is less human-readable..


In [ ]:
!cat pplace.biom

However, there is resources like [phinch](http://phinch.org/index.html) out there to vizualize the contents of biom tables. 

__GIVE IT A TRY__ and explore the results of this analysis in [phinch](http://phinch.org/index.html).

#WELL DONE!#

